### 加载环境和基础设置

In [14]:
import os
import sys
import time
import logging
import glob
import subprocess
import pandas as pd

from tqdm import tqdm

from RootReader import RootData

# 切换到当前目录，并获取工作目录
os.chdir(sys.path[0])
current_path = os.path.abspath(os.curdir)

# 定义相对路径
rel_dir_root = r'./Data/RootData/'
rel_dir_csv  = r'./Data/CSVData/'
rel_dir_mac  = r'./Data/MacLog/'
rel_dir_log  = r'./Data/RunLog/'
rel_dir_geant4 = r'../../build/CompScintSim'

# 获取绝对路径，末尾加上 '/'（注意：Windows下可不加，但在 Linux 下统一加斜杆更清晰）
abs_dir_root   = os.path.abspath(os.path.join(current_path, rel_dir_root)) + '/'
abs_dir_csv    = os.path.abspath(os.path.join(current_path, rel_dir_csv)) + '/'
abs_dir_mac    = os.path.abspath(os.path.join(current_path, rel_dir_mac)) + '/'
abs_dir_log    = os.path.abspath(os.path.join(current_path, rel_dir_log)) + '/'
abs_dir_geant4 = os.path.abspath(os.path.join(current_path, rel_dir_geant4))

print("RootDir:", abs_dir_root)
print("CSVDir:", abs_dir_csv)
print("MacDir:", abs_dir_mac)
print("LogDir:", abs_dir_log)
print("Geant4Dir:", abs_dir_geant4)

# 确保各个目录存在
for directory in [abs_dir_root, abs_dir_csv, abs_dir_mac, abs_dir_log]:
    os.makedirs(directory, exist_ok=True)

# 清除已有日志处理器（避免重复配置）
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# 配置日志记录，日志保存在 abs_dir_log 下
log_filename = "0batch_process.log"
logging.basicConfig(filename=os.path.join(abs_dir_log, log_filename),
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("初始化运行日志")

# 设置全局变量，记录模拟开始前的文件列表
global_before_files = set(os.listdir(abs_dir_root))

class MySim:
    def __init__(self, particle: str = None, energy: str = None, number: str = None, root_file: str = None, profile: str = None):
        if profile:
            # 首先按下划线分割
            parts = profile.split('_')
            # 检查是否至少有3个部分（粒子、能量、数量）
            if len(parts) >= 3 and parts[1].endswith("MeV"):
                self.particle = parts[0]
                self.energy = parts[1][:-3]  # 去掉 "MeV"
                # 取第三部分作为数量，其余部分作为后缀
                self.number = parts[2]
                # 如果有额外的部分，将它们作为后缀重新组合
                self.suffix = '_' + '_'.join(parts[3:]) if len(parts) > 3 else ''
            else:
                raise ValueError("Invalid profile format. Expected format: 'particle_energyMeV_number'")
        else:
            if particle is None or energy is None or number is None:
                raise ValueError("particle, energy, and number must be provided if profile is not used.")
            self.particle = particle
            self.energy = energy
            self.number = number
            self.suffix = ''
        
        self.run_profile = f"{self.particle}_{self.energy}MeV_{self.number}{self.suffix}"
        
        if root_file is None:
            self.root_file = f"{abs_dir_root+self.run_profile}.root"
        else:
            self.root_file = root_file
        
        self.mac_content = generate_mac(self.particle, self.energy, self.number, self.root_file)

    
    def get_profile(self):
        return self.run_profile
    
    def write_mac_file(self, dir_mac, mac_name):
        dir_mac_file = os.path.join(dir_mac, mac_name)
        # 检查目录是否存在
        if not os.path.exists(dir_mac):
            raise FileNotFoundError(f"目录不存在：{dir_mac}")
        # 如果文件已存在，自动重命名增加6位时间戳
        if os.path.exists(dir_mac_file):
            print(f"文件已存在：{dir_mac_file}")
            current_time = time.localtime()
            timestamp = time.strftime("%H%M%S", current_time)
            print(f"自动重命名为：{dir_mac_file}_{timestamp}")
            dir_mac_file = f"{dir_mac_file}_{timestamp}"
        with open(dir_mac_file, "w") as f:
            f.write(self.mac_content)
        return dir_mac_file

def generate_mac(particle: str, energy: str, number: str, root_file: str):
    def validate_params(particle: str, number: str):
        valid_particles = ["proton", "e-", "e+", "gamma"]
        if particle not in valid_particles:
            raise ValueError(f"无效的粒子类型：{particle}")
        if not number.isdigit():
            raise ValueError(f"number 不是整数：{number}")
        return particle, int(number)
    
    validate_params(particle, number)
    template_mac = f'''/control/verbose 0
/tracking/verbose 0
/run/verbose 0
/control/cout/ignoreThreadsExcept 0
/run/initialize
/CompScintSim/generator/useParticleGun true
/MySim/setSaveName {root_file}
/gun/particle {particle}
/gun/energy {energy} MeV
/run/beamOn {number}
'''
    mac_content = template_mac.format(particle=particle, energy=energy, number=number, root_file=root_file)
    return mac_content

def run_geant4(dir_geant4, dir_mac_file, log_file, retries=3):
    """
    运行 Geant4 模拟，并记录批处理日志：
      - Geant4 的标准输出和错误输出全部写入日志文件 {abs_dir_log}{log_file}.log
      - 记录模拟是否成功、耗时、生成的新文件等信息，以及使用的 mac 文件名称
      - 如果新生成的 .root 文件名中包含 '_t0' ，说明模拟失败，
        则删除所有 *_t*.root 文件并重新运行模拟（允许重试最多 3 次）
    """
    global global_before_files

    logging.info("##########################################")
    logging.info("开始运行 Geant4 模拟，使用的 mac 文件：%s", dir_mac_file)
    start_time = time.time()
    
    # 设置日志文件路径
    simulation_log_file = os.path.join(abs_dir_log, f"{log_file}.log")
    
    # 运行 Geant4 并将 stdout 和 stderr 重定向到日志文件
    with open(simulation_log_file, "w") as log:
        cmd = [dir_geant4, "-m", dir_mac_file]
        result = subprocess.run(cmd, stdout=log, stderr=log)  # 捕获所有输出到日志文件

        if result.returncode != 0:
            logging.error("Geant4 运行过程中返回非零状态码，可能存在错误，详情见 %s", simulation_log_file)

    elapsed_time = time.time() - start_time
    after_files = set(os.listdir(abs_dir_root))
    new_files = list(after_files - global_before_files)

    logging.info("模拟运行完成，耗时：%.2f 秒", elapsed_time)
    logging.info("生成的新文件：%s", new_files)

    # 检查是否有输出文件包含 '_t0' 且以 .root 结尾（模拟失败的标志）
    failed = any("_t0" in f and f.endswith(".root") for f in new_files)

    if failed:
        logging.error("检测到输出文件中包含 '_t0'，模拟失败。")
        # 删除所有 *_t*.root 文件
        pattern = os.path.join(abs_dir_root, "*_t*.root")
        for file in glob.glob(pattern):
            os.remove(file)
            logging.info("删除失败输出文件：%s", file)
        if retries > 0:
            logging.info("开始重新运行模拟...剩余重试次数：%d", retries)
            return run_geant4(dir_geant4, dir_mac_file, log_file, retries=retries-1)
        else:
            logging.error("重试后模拟仍失败。")
    else:
        logging.info("模拟成功！使用的 mac 文件：%s；耗时：%.2f 秒；生成的新文件：%s", dir_mac_file, elapsed_time, new_files)

    return new_files



RootDir: /home/wsl2/myGeant4/LightCollect/scripts/auto_python/Data/RootData/
CSVDir: /home/wsl2/myGeant4/LightCollect/scripts/auto_python/Data/CSVData/
MacDir: /home/wsl2/myGeant4/LightCollect/scripts/auto_python/Data/MacLog/
LogDir: /home/wsl2/myGeant4/LightCollect/scripts/auto_python/Data/RunLog/
Geant4Dir: /home/wsl2/myGeant4/LightCollect/build/CompScintSim


### 测试输出

In [8]:
global_before_files = set(os.listdir(abs_dir_root))
test_name = "test"
test_mac = MySim(particle="proton", energy="10", number="1000", root_file=abs_dir_root+test_name + ".root")
abs_dir_mac_file = test_mac.write_mac_file(abs_dir_mac, test_name + ".mac")
run_geant4(abs_dir_geant4, abs_dir_mac_file, test_name)
RootData(os.path.join(abs_dir_root+test_name+".root")).save_layers_mean(abs_dir_csv+ test_name+".csv")

test_profile = 'e-_0.02MeV_10000'
test_mac2 = MySim(profile = test_profile)
abs_dir_mac_file = test_mac2.write_mac_file(abs_dir_mac, test_profile + ".mac")
run_geant4(abs_dir_geant4, abs_dir_mac_file, test_profile)
RootData(os.path.join(abs_dir_root+test_profile+".root")).save_layers_mean(abs_dir_csv+ test_profile+".csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/wsl2/myGeant4/LightCollect/scripts/auto_python/Data/RootData/'

### 实际使用

In [ ]:
# 从0.01到2 MeV，步长为0.1 MeV
energyList = ["1", "1.5", "2"]
# 或者使用列表推导式生成0.01到2的列表
# energyList = [str(round(i * 0.1, 2)) for i in range(1, 21)]

particleList = ["e-"]
numberList = ["100"]

global_before_files = set(os.listdir(abs_dir_root))

# 设置尾缀
suffix = "_test"

# 遍历所有组合，生成profile列表
profileList = []
for particle in particleList:
    for energy in energyList:
        for number in numberList:
            profile = f"{particle}_{energy}MeV_{number}" + suffix
            profileList.append(profile)
            
scintillation_data = []
fiber_entry_data = []
fiber_numerical_aperture_data = []
total_profiles = len(profileList)
with tqdm(total=total_profiles, desc="Processing Profiles") as pbar:
    for profile in profileList:
        sim_instance = MySim(profile=profile)
        abs_dir_mac_file = sim_instance.write_mac_file(abs_dir_mac, profile + ".mac")
        run_geant4(abs_dir_geant4, abs_dir_mac_file, profile)
        root_file = os.path.join(abs_dir_root + profile + ".root")
        root_data = RootData(root_file)
        root_data.save_layers_mean(abs_dir_csv + profile + ".csv")
        scintillation_data.append(root_data.get_h1_data('H1_1')[0].sum())
        fiber_numerical_aperture_data.append(root_data.get_h1_data('H1_3')[0].sum())
        fiber_entry_data.append(root_data.get_h1_data('H1_4')[0].sum())
        pbar.update(1)

# 保存光收集数据到csv文件
data = {
    'name': profileList,
    'scintillation': scintillation_data,
    'fiber_entry': fiber_entry_data,
    'fiber_numerical_aperture': fiber_numerical_aperture_data
}
df = pd.DataFrame(data)
df.to_csv(abs_dir_csv + 'light_collect_data.csv', index=False)




Processing Profiles:   0%|          | 0/3 [00:00<?, ?it/s]

Processing Profiles: 100%|██████████| 3/3 [00:31<00:00, 10.34s/it]
